# Test Structured Amazon Bedrock Knowledge Base

This notebook demonstrates testing the structured Amazon Bedrock Knowledge Base created in the previous notebook using the `RetrieveAndGenerate` API. 

## Data Schema Overview

Our e-commerce database contains the following tables:
- **orders**: order_id, customer_id, order_total, order_status, payment_method, shipping_address, created_at, updated_at
- **order_items**: order_item_id, order_id, product_id, quantity, price
- **reviews**: review_id, product_id, customer_id, rating, created_at
- **payments**: payment_id, order_id, customer_id, amount, payment_method, payment_status, created_at


## Setup and Prerequisites

### Prerequisites
* Completed `2.1-prerequisites-structured-kb.ipynb` notebook
* Structured Knowledge Base successfully created and synced 

Let's start by importing the required libraries and loading the Knowledge Base configuration:


Import required libraries for AWS service interaction and testing the structured Knowledge Base query capabilities.

In [ ]:
import json
import logging
import os
import time
from datetime import datetime

import boto3

Initialize AWS service clients for Bedrock Agent Runtime to query the structured Knowledge Base and retrieve account information.

In [ ]:
session = boto3.session.Session()
region = session.region_name
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]
# Initialize the bedrock agent runtime client
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime")

print(f"AWS Region: {region}")
print(f"AWS Account ID: {account_id}")

## Load Structured Knowledge Base Configuration

Load the Knowledge Base ID and configuration from the prerequisite notebook.


In [ ]:
# Load stored variables from previous notebook
%store -r structured_kb_id
%store -r structured_kb_region
%store -r structured_workgroup_arn
%store -r structured_database_name

# Validate that variables were loaded
if 'structured_kb_id' not in locals():
    raise ValueError("Structured KB ID not found. Please run 2.1-prerequisites-structured-kb.ipynb first.")

print("Structured Knowledge Base Configuration:")
print("=" * 50)
print(f"KB ID: {structured_kb_id}")
print(f"Region: {structured_kb_region}")
print(f"Workgroup ARN: {structured_workgroup_arn}")
print(f"Database: {structured_database_name}")
print("=" * 50)


## Set up Bedrock Foundation Model

Here we configure the Bedrock foundation model, which will be used by the `RetrieveAndGenerate` API to generate answers based on retrieved knowledge base content.

In [ ]:
foundation_model = "global.anthropic.claude-haiku-4-5-20251001-v1:0"

print(f"Using foundation model: {foundation_model}")

## Helper Functions

Define helper functions to format and display results from the Knowledge Base APIs.


In [ ]:
def display_rag_results(response, query):
    """Display results from the retrieve_and_generate API, avoiding duplicate SQL queries."""
    print(f"\nQUERY: {query}")
    print("-" * 40)
    
    if 'output' in response:
        print("GENERATED RESPONSE:")
        print("-" * 40)
        print(response['output']['text'])
        
        if 'citations' in response:
            citations = response['citations']
            
            # Track seen SQL queries to avoid repeats
            seen_sql_queries = set()
            
            for i, citation in enumerate(citations, 1):
                print(f"\nCitation {i}:")
               
                references = []
                if 'retrievedReferences' in citation:
                    references = citation['retrievedReferences']
                elif 'content' in citation and 'location' in citation:
                    # Wrap top-level citation as a single reference for uniformity
                    references = [citation]
                else:
                    references = []
                for j, ref in enumerate(references, 1):
                    # Try to print SQL query if present
                    sql_query = None
                    # Check for SQL location in reference
                    location = ref.get('location', {})
                    if 'sqlLocation' in location:
                        sql_query = location['sqlLocation'].get('query', None)
                    # Sometimes, for top-level citation, location is at citation level
                    elif 'sqlLocation' in citation.get('location', {}):
                        sql_query = citation['location']['sqlLocation'].get('query', None)
                    if sql_query:
                        if sql_query not in seen_sql_queries:
                            print(f"    SQL Query: {sql_query}")
                            seen_sql_queries.add(sql_query)
                        # else: do not print anything for duplicates
                    else:
                        print(f"    SQL Query: (not found)")
    else:
        print("No response generated")

def run_structured_kb_query(query):
    """Test a query using only the retrieve_and_generate API"""

    try:
        rag_response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={'text': query},
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': structured_kb_id,
                    "modelArn": f"arn:aws:bedrock:{region}:{account_id}:inference-profile/{foundation_model}",
                    "retrievalConfiguration": {
                        "vectorSearchConfiguration": {
                            "numberOfResults": 5
                        }
                    }
                }
            }
        )
        display_rag_results(rag_response, query)
    except Exception as e:
        print(f"Error in retrieve_and_generate API: {str(e)}")


## Test Queries for Structured Data

Let's test queries to ensure the Knowledge Base works to our baseline expectations. These queries will demonstrate how the Knowledge Base performs with minimal configuration.

**Why test now?** It's important to validate that each knowledge base works correctly before integrating it into the full agent system. Testing in isolation makes it easier to debug and reconfigure without navigating the complexity of the complete agentic system.


### Query 1

Lets check that the review count can be correctly retrieved from our redshift database.

The [reviews.csv](../sample_structured_data/reviews.csv) has 10,000 entries that should have been loaded into the database during our setup steps

In [ ]:
run_structured_kb_query("How many reviews do we have in total?")


### Query 2

Our dataset has product_553 with 5 1-star ratings.  Lets test this. 

In [ ]:
run_structured_kb_query("Which product_ids received the most 1-star ratings in 2022?")

## Next Steps

If all tests completed successfully, your structured Knowledge Base is working correctly!

**Ready to continue?** Proceed to [Lab 3](../Lab%203%20-%20AgentCore%20Deployment%20and%20Memory/3.1-agentcore-deployment.ipynb) to deploy your intelligent RAG agent to Amazon Bedrock AgentCore Runtime with memory capabilities and observability.
